In [68]:
import torch
import torch.nn as nn
import torch.optim as optim

In [69]:

import pandas as pd

data = pd.read_csv("customers.csv")

data.head()

,Gender_x,Age,Profession,Work_Experience,Family_Size,Spending_Score,Cost_of_the_Product,Prior_purchases,Discount_offered
0,Male,22,Healthcare,1.0,4.0,Low,177,3,44
1,Female,38,Engineer,0.0,3.0,Average,216,2,59
2,Female,67,Engineer,1.0,1.0,Low,183,4,48
3,Male,67,Lawyer,0.0,2.0,High,176,4,10
4,Female,40,Entertainment,0.0,6.0,High,184,3,46


In [70]:
# string to int classification
from sklearn.preprocessing import LabelEncoder


label_encoder = LabelEncoder()
string_fields = ['Gender_x', 'Profession', 'Spending_Score']
for string_field in string_fields:
  data[string_field] = label_encoder.fit_transform(data[string_field])

data.head()

,Gender_x,Age,Profession,Work_Experience,Family_Size,Spending_Score,Cost_of_the_Product,Prior_purchases,Discount_offered
0,1,22,5,1.0,4.0,2,177,3,44
1,0,38,2,0.0,3.0,0,216,2,59
2,0,67,2,1.0,1.0,2,183,4,48
3,1,67,7,0.0,2.0,1,176,4,10
4,0,40,3,0.0,6.0,1,184,3,46


In [71]:
# data normalization
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

for field_name in data.columns:
  data[field_name] = scaler.fit_transform(data[[field_name]])

print(data)


      Gender_x       Age  Profession  Work_Experience  Family_Size  \
0          1.0  0.056338       0.625         0.071429     0.444444   
1          0.0  0.281690       0.250         0.000000     0.333333   
2          0.0  0.690141       0.250         0.071429     0.111111   
3          1.0  0.690141       0.875         0.000000     0.222222   
4          0.0  0.309859       0.375         0.000000     0.666667   
...        ...       ...         ...              ...          ...   
7939       1.0  0.323944       0.000         0.000000     0.555556   
7940       1.0  0.239437       0.500         0.214286     0.444444   
7941       0.0  0.211268       0.625         0.071429     0.111111   
7942       0.0  0.126761       0.625         0.071429     0.444444   
7943       1.0  0.267606       0.500         0.000000     0.333333   

      Spending_Score  Cost_of_the_Product  Prior_purchases  Discount_offered  
0                1.0             0.378505            0.125          0.671875  
1

In [72]:
# test training splitting
from sklearn.model_selection import train_test_split

y_cols = ['Cost_of_the_Product', "Spending_Score"]

X = data.drop(columns=y_cols)
y = data[y_cols]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train = torch.tensor(X_train.values)
X_test = torch.tensor(X_test.values)
y_train = torch.tensor(y_train.values)
y_test = torch.tensor(y_test.values)

X_train = X_train.to(torch.float32)
X_test = X_test.to(torch.float32)
y_train = y_train.to(torch.float32)
y_test = y_test.to(torch.float32)

print("Training set shape:", X_train.shape, y_train.shape)
print("Testing set shape:", X_test.shape, y_test.shape)
print(X_train.dtype)
print(X_test.dtype)
print(y_train.dtype)
print(y_test.dtype)

Training set shape: torch.Size([6355, 7]) torch.Size([6355, 2])
Testing set shape: torch.Size([1589, 7]) torch.Size([1589, 2])
torch.float32
torch.float32
torch.float32
torch.float32


In [73]:
# NN
class SpendingsPredictor(nn.Module):
    def __init__(self, input_size: int, dropout_p: float = 0.5):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 2)
        )

    def forward(self, x):
        return self.mlp(x)

    def predict_proba(self, x):
        return sigmoid(self(x))

    def predict(self, x):
        y_pred_score = self.predict_proba(x)
        return torch.argmax(y_pred_score, dim=1)

In [74]:
# training configs

from copy import deepcopy

from torch.utils.data import DataLoader


learning_rate = 1e-3
dropout_p = 0.5
l2_reg = 1e-4
max_epochs = 300_000


early_stopping_patience = 10

In [75]:
model = SpendingsPredictor(
    input_size=X_train.shape[1],
    dropout_p=dropout_p
)
optimizer = torch.optim.SGD(
    model.parameters(),
    lr=learning_rate,
    weight_decay=l2_reg
)

loss_fn = nn.MSELoss()

train_dataset = MyDataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size)


In [76]:
steps_without_improvement = 0

best_val_loss = np.inf
best_model = None

for epoch_num in range(max_epochs):
    model.train()

    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    model.eval()

    if loss < best_val_loss:
        best_val_loss = loss
        best_model = deepcopy(model)
        steps_without_improvement = 0
    else:
        steps_without_improvement += 1

    print(f"Epoch {epoch_num} train loss: {loss.item():.4f}")

    if steps_without_improvement >= early_stopping_patience:
        print(f"Early stopping after {epoch_num} epochs.")
        break

Epoch 0 train loss: 0.6270
Epoch 1 train loss: 0.6230
Epoch 2 train loss: 0.6194
Epoch 3 train loss: 0.6177
Epoch 4 train loss: 0.6155
Epoch 5 train loss: 0.6084
Epoch 6 train loss: 0.6063
Epoch 7 train loss: 0.6004
Epoch 8 train loss: 0.5981
Epoch 9 train loss: 0.5926
Epoch 10 train loss: 0.5927
Epoch 11 train loss: 0.5859
Epoch 12 train loss: 0.5851
Epoch 13 train loss: 0.5802
Epoch 14 train loss: 0.5769
Epoch 15 train loss: 0.5752
Epoch 16 train loss: 0.5710
Epoch 17 train loss: 0.5664
Epoch 18 train loss: 0.5640
Epoch 19 train loss: 0.5606
Epoch 20 train loss: 0.5584
Epoch 21 train loss: 0.5561
Epoch 22 train loss: 0.5508
Epoch 23 train loss: 0.5490
Epoch 24 train loss: 0.5443
Epoch 25 train loss: 0.5424
Epoch 26 train loss: 0.5404
Epoch 27 train loss: 0.5356
Epoch 28 train loss: 0.5329
Epoch 29 train loss: 0.5331
Epoch 30 train loss: 0.5260
Epoch 31 train loss: 0.5232
Epoch 32 train loss: 0.5205
Epoch 33 train loss: 0.5202
Epoch 34 train loss: 0.5159
Epoch 35 train loss: 0.5127
Ep